In [1]:
import sys
sys.path.append("..")
from dataset import load_polaris_dataset, validate_dataset
from train import get_dataset
import numpy as np
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    PreTrainedTokenizer
)
import torch
from peft import PeftModel
from peft import prepare_model_for_kbit_training
from trl import ModelConfig
from munch import Munch
import json
from pathlib import Path
from functools import partial
import hashlib
from collections import defaultdict

/home/alisavin/AgenticADMET/openr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 03-02 03:25:52 __init__.py:190] Automatically detected platform cuda.


2025-03-02 03:25:52,942	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

In [3]:
def compute_mae(completions, ground_truth=None, log_normalize=False, model_name="pred", **kwargs):
    
    smiles = kwargs.get("smiles")
    num_generations = len(completions) / len(set(smiles))
    
    Path(f"./test/completions/{model_name}_{num_generations}/").mkdir(parents=True, exist_ok=True)
    
    contents = [completion[0]["content"] for completion in completions]
    rewards = []

    solutions = kwargs.get("solution") # Get solutions from kwargs
    

    if solutions is None:
        return [0.5] * len(completions) # Return neutral reward if no solution
    smiles2conts = defaultdict(list)
    #TODO: not parsed empty list
    #TODO: mae mean per group, take meadian and calculate
    #TODO: make more generations, save separately
    for content, gold_val, smiles_i in zip(contents, solutions, smiles):
        
        if gold_val is not None:  # Check if parsing was successful
            # Parse the model's answer with relaxed normalization
            answer_parsed = parse(
                content,
                extraction_config=[
                    LatexExtractionConfig(
                        normalization_config=NormalizationConfig(
                            nits=False,
                            malformed_operators=False,
                            basic_latex=True,
                            equations=True,
                            boxed="all",
                            units=True,
                        ),
                        boxed_match_priority=0,
                        try_extract_without_anchor=False,
                    )
                ],
                extraction_mode="first_match",
            )

            try:
                answer_val = float(answer_parsed[0])
                mae = np.mean(np.abs(gold_val - answer_val))
                print("parsed correctly", answer_val, gold_val)
            except Exception as e:
                answer_val = None
                mae = 6
                if len(answer_parsed) > 0:
                    print(e, answer_parsed)
        else:
            answer_val = None
            mae = 6
            print("Warning: Gold solution is None:", gold_val)
        if answer_val is not None:
            post = "parsed_"
        else:
            post = ""
        smiles_hash = hashlib.blake2b(smiles_i.encode('utf-8'), digest_size=4).hexdigest()
        rewards.append(mae)
        smiles2conts[smiles_hash].append({"completion": content, 
                       "gold_val": str(gold_val), 
                       "answer_parsed": str(answer_parsed), 
                       "smiles": smiles_i,
                       "answer_val": answer_val,
                       "mae": mae
                       }) 
    median_maes = []
    for k, v in smiles2conts.items():
        with open(f"./test/completions/{model_name}_{num_generations}/{post}{k}.json", "w") as f:
                answers_g = [v_i["answer_val"] for v_i in v]
                answers_g = [float(v_i) for v_i in answers_g if v_i is not None]
                answer_median = np.median(answers_g)
                mae_median = np.median(np.abs(float(v[0]["gold_val"]) - answer_median))
                median_maes.append(mae_median)
                json.dump({"completion": [v_i["completion"] for v_i in v], 
                        "gold_val": v[0]["gold_val"], 
                        "answer_parsed": [v_i["answer_parsed"] for v_i in v], 
                        "smiles": v[0]["smiles"],
                        "answer_val": [v_i["answer_val"] for v_i in v],
                        "mae": [v_i["mae"] for v_i in v],
                        "mae_median": str(mae_median)
                        }, f, indent=2)
    return median_maes

def get_tokenizer(
    model_args: ModelConfig, training_args, auto_set_chat_template: bool = True
) -> PreTrainedTokenizer:
    """Get the tokenizer for the model."""
    # https://github.com/huggingface/open-r1/blob/eeca246b078457bc0f69ba2e8297b799df0e2bda/src/open_r1/utils/model_utils.py#L11
    print("loading tokenizer")
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        revision=model_args.model_revision,
        trust_remote_code=False, # model_args.trust_remote_code
    )
    print("tokenizer loaded")

    if training_args.chat_template is not None:
        tokenizer.chat_template = training_args.chat_template
    elif auto_set_chat_template and tokenizer.get_chat_template() is None:
        tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE
    print("chat template")
    # if processing_class is None:
    #     processing_class = AutoTokenizer.from_pretrained(model.config._name_or_path, padding_side="left")
    return tokenizer

In [4]:
dataset = get_dataset(params=["LogD"], subset_train=50)

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Map: 100%|██████████| 52/52 [00:00<00:00, 5059.00 examples/s]

Train set size: 221
Test set size: 49

Validating train split:
✓ All required fields present
✓ Prompt format is correct

Validating test split:
✓ All required fields present
✓ Prompt format is correct


In [5]:
len(dataset["train"]), len(dataset["validation"])

(50, 52)

In [6]:
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

# model = AutoModelForCausalLM.from_pretrained()
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,    # if required
    torch_dtype=torch.bfloat16,  # if you used bf16
    device_map="auto"           # or "cuda:0", depending on your environment
)

# # 2) Load LoRA adapter weights onto the base model
# model = PeftModel.from_pretrained(base_model, "/home/alisavin/AgenticADMET/outputs/2025-02-26/22-18-57/checkpoint-60/")
# model = prepare_model_for_kbit_training(model)
# model = base_model
# model.eval()

model_args_i = Munch.fromDict({
        "model_name_or_path": MODEL_NAME,
        "model_revision": "main",
        "trust_remote_code": False # TODO: everyboudy sets to True and default is True
        })

training_args_i = Munch.fromDict({"chat_template": "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% set ns = namespace(is_first=false, is_tool=false, is_output_first=true, system_prompt='') %}{%- for message in messages %}{%- if message['role'] == 'system' %}{% set ns.system_prompt = message['content'] %}{%- endif %}{%- endfor %}{{bos_token}}{{ns.system_prompt}}{%- for message in messages %}{%- if message['role'] == 'user' %}{%- set ns.is_tool = false -%}{{'<｜User｜>' + message['content']}}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is none %}{%- set ns.is_tool = false -%}{%- for tool in message['tool_calls']%}{%- if not ns.is_first %}{{'<｜Assistant｜><｜tool▁calls▁begin｜><｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\\n' + '```json' + '\\n' + tool['function']['arguments'] + '\\n' + '```' + '<｜tool▁call▁end｜>'}}{%- set ns.is_first = true -%}{%- else %}{{'\\n' + '<｜tool▁call▁begin｜>' + tool['type'] + '<｜tool▁sep｜>' + tool['function']['name'] + '\\n' + '```json' + '\\n' + tool['function']['arguments'] + '\\n' + '```' + '<｜tool▁call▁end｜>'}}{{'<｜tool▁calls▁end｜><｜end▁of▁sentence｜>'}}{%- endif %}{%- endfor %}{%- endif %}{%- if message['role'] == 'assistant' and message['content'] is not none %}{%- if ns.is_tool %}{{'<｜tool▁outputs▁end｜>' + message['content'] + '<｜end▁of▁sentence｜>'}}{%- set ns.is_tool = false -%}{%- else %}{% set content = message['content'] %}{{'<｜Assistant｜>' + content + '<｜end▁of▁sentence｜>'}}{%- endif %}{%- endif %}{%- if message['role'] == 'tool' %}{%- set ns.is_tool = true -%}{%- if ns.is_output_first %}{{'<｜tool▁outputs▁begin｜><｜tool▁output▁begin｜>' + message['content'] + '<｜tool▁output▁end｜>'}}{%- set ns.is_output_first = false %}{%- else %}{{'\\n<｜tool▁output▁begin｜>' + message['content'] + '<｜tool▁output▁end｜>'}}{%- endif %}{%- endif %}{%- endfor -%}{% if ns.is_tool %}{{'<｜tool▁outputs▁end｜>'}}{% endif %}{% if add_generation_prompt and not ns.is_tool %}{{'<｜Assistant｜>'}}{% endif %}"})

tokenizer = get_tokenizer(model_args_i, training_args_i)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# # 4) Generate text from a prompt
# prompt = "Explain the concept of molecular solubility in simple terms."

# input = {
#         "ground_truth": example["solution"],
#         "prompt": [
#             {"role": "system", "content": SYSTEM_PROMPT},
#             {"role": "user", "content": example["problem"]},
#         ],
#     }
# inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# with torch.no_grad():
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=128,
#         do_sample=True,
#         temperature=0.7,
#         top_p=0.9
#     )

# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print("Generated:", generated_text)
from train import GRPOTrainer2
import os
from trl import (
    GRPOConfig, 
    GRPOTrainer,
    get_peft_config
)
from dataclasses import field, dataclass

def get_reward_functions(script_args, model_name):
    """
    Returns a list of reward functions based on the script arguments.
    """
    reward_funcs_list = []

    fnc = partial(compute_mae, model_name=model_name)
    fnc.__name__ = compute_mae.__name__
    reward_funcs_registry = {
        "mae": fnc,  # Assuming accuracy_reward is defined in previous steps
    }

    for func_name in script_args.reward_funcs:
        if func_name not in reward_funcs_registry:
            raise ValueError(f"Reward function '{func_name}' not found in registry.")
        reward_funcs_list.append(reward_funcs_registry[func_name])

    return reward_funcs_list

@dataclass
class GRPOScriptArguments:
    """
    Script arguments for GRPO training, specifically related to reward functions.
    """

    reward_funcs: list[str] = field(
        default_factory=lambda: ["mae"], 
        metadata={
            "help": "List of reward functions. Possible values: 'accuracy', 'format', 'reasoning_steps', 'repetition_penalty'"        },
    )

    repetition_n_grams: int = field(
        default=3,
        metadata={"help": "Number of n-grams for repetition penalty reward"},
    )
    repetition_max_penalty: float = field(
        default=-0.1,
        metadata={"help": "Maximum (negative) penalty for for repetition penalty reward"},
    )
    
script_args = GRPOScriptArguments()

reward_functions = get_reward_functions(script_args, model_name="tuned_v3_correct_format_v2") #TODO: check trl they had someshere gpro example and used different rewards including lenght reward

# training_args = TrainingArguments(
#     logging_dir="./logs/wandb/",
#     num_train_epochs=11,             # Total number of training epochs
#     per_device_train_batch_size=16,  # Batch size per device during training
#     per_device_eval_batch_size=16,   # Batch size for evaluation TODO: why it says this   File "/home/alisavin/AgenticADMET/train.py", line 534, in <module>
#     gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
#     learning_rate=1e-6,            # Initial learning rate for AdamW optimizer
#     warmup_ratio=0.1,              # Linear warmup over warmup_ratio fraction of training steps
#     weight_decay=0.01,             # Apply weight decay to all layers except bias and LayerNorm weights
#     logging_steps=1,              # Log every X updates steps
#     logging_strategy="steps",
#     logging_first_step=True,
#     evaluation_strategy="epoch",    # Evaluate every `eval_steps`
#     save_strategy="no",      # Disables regular checkpoints
#     save_total_limit=0,      # Makes sure no checkpoints are kept
#     load_best_model_at_end=False,  # Disables saving the best model
#     dataloader_num_workers=4,      # Number of subprocesses to use for data loading
#     seed=42,                       # Random seed for reproducibility
#     bf16=True,                     # Use mixed precision BFP16 training #TODO: ??????
#     push_to_hub=False,             # Whether to push the final model to Hugging Face Hub
#     report_to=["wandb"],              # Reporting to no one
#     run_name="test",
#     do_train=False,
#     disable_tqdm=False,
#     gradient_checkpointing=True,   # Enable gradient checkpointing        
#     remove_unused_columns=False,
#     do_eval=True, #TODO: use
#     gradient_checkpointing_kwargs={"use_reentrant": False}, # TODO: use
#     lr_scheduler_type="cosine_with_min_lr",
#     lr_scheduler_kwargs={"min_lr_rate": 0.1},
#     max_steps=-1, #TODO: change to -1
#     resume_from_checkpoint="/home/alisavin/AgenticADMET/outputs/2025-02-26/22-18-57/checkpoint-60/"
# )

training_args = TrainingArguments(
        logging_dir="./logs/wandb/",
        num_train_epochs=12,             # Total number of training epochs
        per_device_train_batch_size=16,  # Batch size per device during training
        per_device_eval_batch_size=16,   # Batch size for evaluation TODO: why it says this   File "/home/alisavin/AgenticADMET/train.py", line 534, in <module>
        gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
        learning_rate=1e-6,            # Initial learning rate for AdamW optimizer
        warmup_ratio=0.1,              # Linear warmup over warmup_ratio fraction of training steps
        weight_decay=0.01,             # Apply weight decay to all layers except bias and LayerNorm weights
        logging_steps=1,              # Log every X updates steps
        logging_strategy="steps",
        logging_first_step=True,
        evaluation_strategy="epoch",    # Evaluate every `eval_steps`
        save_strategy="no",      # Disables regular checkpoints
        save_total_limit=0,      # Makes sure no checkpoints are kept
        load_best_model_at_end=False,  # Disables saving the best model
        dataloader_num_workers=4,      # Number of subprocesses to use for data loading
        seed=42,                       # Random seed for reproducibility
        bf16=True,                     # Use mixed precision BFP16 training #TODO: ??????
        push_to_hub=False,             # Whether to push the final model to Hugging Face Hub
        report_to=["wandb"],              # Reporting to no one
        run_name="test",
        disable_tqdm=False,
        gradient_checkpointing=True,   # Enable gradient checkpointing        
        remove_unused_columns=False,
        do_train=True,
        # do_eval=True, #TODO: use
        gradient_checkpointing_kwargs={"use_reentrant": False}, # TODO: use
        lr_scheduler_type="cosine_with_min_lr",
        lr_scheduler_kwargs={"min_lr_rate": 0.1},
        max_steps=-1, #TODO: change to -1
    )

grpo_config = GRPOConfig(
    **training_args.to_dict(), # Convert TrainingArguments to dictionary and unpack
    **{ 
    # REMOVED model_init_kwargs here 
    # We are passing the instantiated 'model' object, so GRPOTrainer doesn't need model_init_kwargs
    },
    num_generations=8, #TODO: 16
    use_vllm=True, #TODO: use True
    vllm_device="cuda:0",
    vllm_gpu_memory_utilization=0.25, # TODO: 0.25 0.7
    vllm_max_model_len=2048, #TODO: 2048
    max_prompt_length=800, #TODO: 800+
    max_completion_length=1024, #TODO: 1024+ (better 2048/4048 and more)
    temperature=0.7,
    reward_weights=[1.0]
    )

model_args = ModelConfig(model_name_or_path=MODEL_NAME, use_peft=True)

grpo_trainer = GRPOTrainer2(
    model=model,                      # Our initialized Qwen model
    reward_funcs=reward_functions,    # List of reward functions from previous step
    args=grpo_config,                # GRPOConfig (created from TrainingArguments)
    train_dataset=dataset['train'],   # Training dataset
    eval_dataset=dataset['validation'],    # Evaluation dataset
    processing_class=tokenizer, #TODO: check callback from config
    peft_config=get_peft_config(model_args) #TODO: check # label_names
    # peft_config=None
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.35s/it]


loading tokenizer
tokenizer loaded
chat template


/home/alisavin/AgenticADMET/openr1/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/alisavin/AgenticADMET/openr1/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/alisavin/AgenticADMET/openr1/lib/python3.11/site-packages/transformers/training_args.py:2077: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used in

In [7]:
# # Before creating the trainer
# if hasattr(model, "active_adapter"):
#     print(f"Active adapter: {grpo_trainer.model.active_adapter}")
#     print("Adapter names:", grpo_trainer.model.peft_config.keys())
# else:
#     print("No adapter is active. This might be just the base model.")

In [8]:
# def are_models_identical(model1, model2):
#     # Get named parameters for both models
#     params1 = dict(model1.named_parameters())
#     params2 = dict(model2.named_parameters())
    
#     # Check if they have the same parameter names
#     if params1.keys() != params2.keys():
#         print("Models have different parameter structures")
#         return False
    
#     # Check if parameter values are identical
#     all_equal = True
#     for name in params1.keys():
#         if not torch.allclose(params1[name], params2[name], atol=1e-5):
#             print(f"Parameters differ at: {name}")
#             all_equal = False
#             # Optional: print some details about the differing parameters
#             print(f"  Model1: min={params1[name].min()}, max={params1[name].max()}, mean={params1[name].mean()}")
#             print(f"  Model2: min={params2[name].min()}, max={params2[name].max()}, mean={params2[name].mean()}")
#             # Only show a few differences to avoid overwhelming output
#             if not all_equal:
#                 break
                
#     return all_equal

# # Usage:
# are_identical = are_models_identical(base_model, model)
# print(f"Models are identical: {are_identical}")

In [10]:
train_result = grpo_trainer.train(resume_from_checkpoint="/home/alisavin/AgenticADMET/outputs/2025-02-26/22-18-57/checkpoint-60/")

/home/alisavin/AgenticADMET/openr1/lib/python3.11/site-packages/transformers/trainer.py:3423: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoin

/home/alisavin/AgenticADMET/openr1/lib/python3.11/site-packages/transformers/trainer.py:3119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.load

parsed correctly 3.8 2.1
parsed correctly 1.85 2.1
parsed correctly 2.5 2.1
parsed correctly 2.8 2.1
parsed correctly 3.5 2.1
parsed correctly 3.2 2.1
parsed correctly 3.5 2.1
parsed correctly 2.8 2.1
parsed correctly 2.5 1.9
parsed correctly 1.25 1.9
parsed correctly 3.5 1.9
parsed correctly 1.3 1.9
parsed correctly 3.8 1.9
parsed correctly 4.7 1.9
parsed correctly 2.5 1.9


RuntimeError: The expanded size of the tensor (16) must match the existing size (2) at non-singleton dimension 0.  Target sizes: [16].  Tensor sizes: [2]

In [11]:
# train_result = grpo_trainer.evaluate()

In [12]:
import glob
from pathlib import Path
import json
import hashlib
import numpy as np

pred_path = "./test/completions/tuned_v3_correct_format_16.0/*.json"
num_generations = Path(pred_path).parts[-2].split("_")[-1]
pths_1 = glob.glob(pred_path)
# pth_2 = glob.glob("test/not_tuned_pred/*json")

dict_all = {}
 
mean_mae_1 = []
mean_mae_2 = []

mean_mae_1_median = []
mean_mae_2_median = []


for pth_i in pths_1:
    with open(pth_i, "r") as f:
        dict_i = json.load(f)
    smiles = dict_i["smiles"]
    smiles_hash = hashlib.blake2b(smiles.encode('utf-8'), digest_size=4).hexdigest()

    pth = f"./test/completions/init_v3_correct_format_{num_generations}/parsed_{smiles_hash}.json" if Path(f"test/completions/init_v3_correct_format_{num_generations}/parsed_{smiles_hash}.json").exists() else f"test/completions/init_v3_correct_format_{num_generations}/{smiles_hash}.json"
    with open(pth, "r") as f:
        dict_i_2 = json.load(f)
    
    print(pth)
    print(pth_i)

    dict_all[smiles] = {
        "7BQwen": {
            "completion": dict_i_2["completion"], 
            "answer_val": dict_i_2["answer_val"],
            "mae": dict_i_2["mae"],
            "mae_median": dict_i_2["mae_median"]
        },
        "7BQwenTuned": {
            "completion": dict_i["completion"], 
            "answer_val": dict_i["answer_val"],
            "mae": dict_i["mae"],
            "mae_median": dict_i["mae_median"]
        },
        "gold_val": dict_i_2["gold_val"],
    }
    if dict_i_2["answer_val"] is None:
        dict_all[smiles]["7BQwen"]["answer_parsed"] = dict_i_2["answer_parsed"]
    if dict_i["answer_val"] is None:
        dict_all[smiles]["7BQwenTuned"]["answer_parsed"] = dict_i_2["answer_parsed"]

    if dict_i["mae"] is not None:
        mean_mae_1.extend([float(v_i) if v_i is not None else 10 for v_i in dict_i["mae"]])
    if dict_i_2["mae"] is not None:
        mean_mae_2.append([float(v_i) if v_i is not None else 10 for v_i in dict_i_2["mae"]])
    print([float(v_i) if v_i is not None else 10 for v_i in dict_i["mae"]][:10])
    print([float(v_i) if v_i is not None else 10 for v_i in dict_i_2["mae"]][:10])
    if dict_i["mae_median"] is not None:
        mean_mae_1_median.append(float(dict_i["mae_median"]))
    if dict_i_2["mae_median"] is not None:
        mean_mae_2_median.append(float(dict_i_2["mae_median"]))
print(f"mean mae tuned - {np.mean(mean_mae_1)}, mean mae - {np.mean(mean_mae_2)}")
print(f"median: mean mae tuned - {np.mean([v_i for v_i in mean_mae_1_median if not np.isnan(v_i)])}, mean mae - {np.mean([v_i for v_i in mean_mae_2_median if not np.isnan(v_i)])}")

with open(f"./test/completions/all_results_v3_{num_generations}.json", "w") as f:
    json.dump(dict_all, f, indent=2)
    


./test/completions/init_v3_correct_format_16.0/parsed_8c64959f.json
./test/completions/tuned_v3_correct_format_16.0/parsed_8c64959f.json
[10, 10, 1.7, 0.9000000000000001, 0.9999999999999998, 2.7, 10, 1.4000000000000001, 0.6000000000000001, 2.9000000000000004]
[10, 10, 1.7, 0.9000000000000001, 0.9999999999999998, 2.7, 10, 1.4000000000000001, 0.6000000000000001, 2.9000000000000004]
./test/completions/init_v3_correct_format_16.0/parsed_beff7dfa.json
./test/completions/tuned_v3_correct_format_16.0/parsed_beff7dfa.json
[2.0999999999999996, 0.5, 0.9000000000000001, 1.7000000000000002, 0.09999999999999964, 10, 10, 0.9000000000000001, 2.5, 3.8]
[2.0999999999999996, 0.5, 0.9000000000000001, 1.7000000000000002, 0.09999999999999964, 10, 10, 0.9000000000000001, 2.5, 3.8]
test/completions/init_v3_correct_format_16.0/829e5d68.json
./test/completions/tuned_v3_correct_format_16.0/829e5d68.json
[10, 0.8, 1.7999999999999998, 0.19999999999999996, 2.0, 1.7999999999999998, 3.8, 0.30000000000000004, 3.2, 10

In [8]:
# import os

# directory = "./test/completions/pred/"  # Replace with your directory path

# for filename in os.listdir(directory):
#     if filename.startswith("parsed"):
#         # Remove the underscore and add "parsed_" if needed
#         new_name = "parsed_" + filename[7:]
        
#         # Full paths for renaming
#         old_path = os.path.join(directory, filename)
#         new_path = os.path.join(directory, new_name)
        
#         # Rename the file
#         os.rename(old_path, new_path)
#         print(f"Renamed: {filename} → {new_name}")

Renamed: parsed3afaf4aa.json → parsed_afaf4aa.json
Renamed: parsedbeff7dfa.json → parsed_eff7dfa.json
Renamed: parsed159fc2ec.json → parsed_59fc2ec.json
Renamed: parsed1a554c5c.json → parsed_a554c5c.json
Renamed: parsedc94a83f1.json → parsed_94a83f1.json
Renamed: parsed189835d9.json → parsed_89835d9.json
Renamed: parsed2bed14c1.json → parsed_bed14c1.json
Renamed: parsed16bad6bb.json → parsed_6bad6bb.json
Renamed: parsedb940e7a8.json → parsed_940e7a8.json
Renamed: parsed21a08268.json → parsed_1a08268.json
Renamed: parsedec93f253.json → parsed_c93f253.json
Renamed: parsed9ef3031d.json → parsed_ef3031d.json
Renamed: parsedfc6cfa4d.json → parsed_c6cfa4d.json
Renamed: parsede8858049.json → parsed_8858049.json
Renamed: parsed03476906.json → parsed_3476906.json
Renamed: parsedd26c74eb.json → parsed_26c74eb.json
Renamed: parsed9698e723.json → parsed_698e723.json
Renamed: parsedbb9f2209.json → parsed_b9f2209.json
Renamed: parsed822656fc.json → parsed_22656fc.json
Renamed: parsedf5f8aded.json → 

In [2]:
import json

with open("../dataset/test_split.json", "r") as f:
    json_dict = json.load(f)

In [4]:
{k: v["LogD"] for k, v in json_dict.items()}

{'CC1=NOC(CN2C[C@@]3(C(=O)N(C4=CN=CC5=CC=CC=C45)C[C@@H]3C)C3=CC(F)=CC=C3C2=O)=C1 |a:8,23|': 2.1,
 'C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(CC1=CNN=N1)C(=O)C1=CC=C(F)C=C12': 1.7,
 'O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H](C2=NN=C3C=CC=CN23)C1 |&1:17|': nan,
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=CC=C3N=NN(C)C3=C1)N2)C1=CC=C2CCCS(=O)(=O)C2=C1 |&1:3|': 3.4,
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=CN(CC(N)=O)N=C1)N2)C1=CC=C2CCCS(=O)(=O)C2=C1 |&1:3|': 1.9,
 'NCC1=CC=CC(NC(=O)C[C@H](NC(=O)OCC2=CC=CC=C2)C2=CC=C(OCC3=CC=CC=C3)C=C2)=C1 |&1:11|': nan,
 'C[C@H](C(=O)O)[C@@H](NC(=O)C1=NC=NC2=C1C=CN2)C1=CC=C(Br)C=C1 |o1:1,5|': 0.6,
 'CN(C)C(=O)C1=NOC(CCCOC2=CC=C3C(OCC4CC4)=NOC3=C2)=C1': 3.7,
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=C(C#N)N(C)N=C1)N2)C1=CC=C2OCC(=O)NC2=C1 |o1:3|': 4.4,
 'C=CC(=O)NC1=CC=CC(N(CC2=CC=CC(Cl)=C2)C(=O)CC2=CN=CC3=CC=CC=C23)=C1': 3.8,
 'CN1C(SC2=CC(C3=CC=CC=C3)=NC3=NC=NN23)=NN=C1C1=CC=NC=C1': 1.4,
 'C=CC(=O)N1CCCC2=CC=C(N(CC3=CC=CC(Cl)=C3)C(=O)CC3=CN=CC4=C(F)C=CC=C34)C=C21': nan,
 'CC(C)[C@H](N

In [8]:
list(json_dict.keys())[:10]

['CC1=NOC(CN2C[C@@]3(C(=O)N(C4=CN=CC5=CC=CC=C45)C[C@@H]3C)C3=CC(F)=CC=C3C2=O)=C1 |a:8,23|',
 'C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(CC1=CNN=N1)C(=O)C1=CC=C(F)C=C12',
 'O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H](C2=NN=C3C=CC=CN23)C1 |&1:17|',
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=CC=C3N=NN(C)C3=C1)N2)C1=CC=C2CCCS(=O)(=O)C2=C1 |&1:3|',
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=CN(CC(N)=O)N=C1)N2)C1=CC=C2CCCS(=O)(=O)C2=C1 |&1:3|',
 'NCC1=CC=CC(NC(=O)C[C@H](NC(=O)OCC2=CC=CC=C2)C2=CC=C(OCC3=CC=CC=C3)C=C2)=C1 |&1:11|',
 'C[C@H](C(=O)O)[C@@H](NC(=O)C1=NC=NC2=C1C=CN2)C1=CC=C(Br)C=C1 |o1:1,5|',
 'CN(C)C(=O)C1=NOC(CCCOC2=CC=C3C(OCC4CC4)=NOC3=C2)=C1',
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=C(C#N)N(C)N=C1)N2)C1=CC=C2OCC(=O)NC2=C1 |o1:3|',
 'C=CC(=O)NC1=CC=CC(N(CC2=CC=CC(Cl)=C2)C(=O)CC2=CN=CC3=CC=CC=C23)=C1']

In [9]:
keys_2 = ['CC1=NOC(CN2C[C@@]3(C(=O)N(C4=CN=CC5=CC=CC=C45)C[C@@H]3C)C3=CC(F)=CC=C3C2=O)=C1 |a:8,23|',
 'C[C@H]1CN(C2=CN=CC3=CC=CC=C23)C(=O)[C@@]12CN(CC1=CNN=N1)C(=O)C1=CC=C(F)C=C12',
 'O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H](C2=NN=C3C=CC=CN23)C1 |&1:17|',
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=CC=C3N=NN(C)C3=C1)N2)C1=CC=C2CCCS(=O)(=O)C2=C1 |&1:3|',
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=CN(CC(N)=O)N=C1)N2)C1=CC=C2CCCS(=O)(=O)C2=C1 |&1:3|',
 'NCC1=CC=CC(NC(=O)C[C@H](NC(=O)OCC2=CC=CC=C2)C2=CC=C(OCC3=CC=CC=C3)C=C2)=C1 |&1:11|',
 'C[C@H](C(=O)O)[C@@H](NC(=O)C1=NC=NC2=C1C=CN2)C1=CC=C(Br)C=C1 |o1:1,5|',
 'CN(C)C(=O)C1=NOC(CCCOC2=CC=C3C(OCC4CC4)=NOC3=C2)=C1',
 'CC(C)[C@H](NC1=NC=NC2=C1C=C(C1=C(C#N)N(C)N=C1)N2)C1=CC=C2OCC(=O)NC2=C1 |o1:3|',
 'C=CC(=O)NC1=CC=CC(N(CC2=CC=CC(Cl)=C2)C(=O)CC2=CN=CC3=CC=CC=C23)=C1']
for k in keys_2:
    print(json_dict[k]["LogD"])

2.1
1.7
nan
3.4
1.9
nan
0.6
3.7
4.4
3.8
